In [4]:
# Install required library (run in notebook once)
!pip install --upgrade google-cloud-aiplatform
# Install / upgrade the SDK (run once)
!pip install --upgrade google-cloud-aiplatform --quiet


# Import libraries
from google.cloud import aiplatform
import pickle

# Initialize Vertex AI
aiplatform.init(project="gen-lang-client-0708250845", location="us-central1")

# Memory Bank for storing student progress
try:
    with open("memory_bank.pkl", "rb") as f:
        memory_bank = pickle.load(f)
except:
    memory_bank = {}

# Function to call Google Vertex AI Generative Model
def google_ai_chat(prompt):
    model = aiplatform.gapic.ModelServiceClient().model_path(
        project="YOUR_GOOGLE_CLOUD_PROJECT_ID",
        location="us-central1",
        model="text-bison@001"  # Built-in text model
    )
    
    # Use the aiplatform client TextGenerationModel instead
    text_model = aiplatform.TextGenerationModel.from_pretrained("text-bison@001")
    response = text_model.predict(
        prompt,
        max_output_tokens=500
    )
    return response.text

# Topic Explainer Agent
def explain_topic(student_id, topic):
    prompt = f"Explain the topic '{topic}' in simple language with examples."
    explanation = google_ai_chat(prompt)
    memory_bank[student_id] = memory_bank.get(student_id, {})
    memory_bank[student_id]["last_topic"] = topic
    memory_bank[student_id]["last_explanation"] = explanation
    return explanation

# Practice Question Agent
def generate_questions(topic):
    prompt = f"Generate 3 practice questions with answers for '{topic}'."
    return google_ai_chat(prompt)

# Feedback Agent
def provide_feedback(student_id, answers):
    last_topic = memory_bank[student_id]["last_topic"]
    prompt = f"Student answers: {answers}. Give feedback, correct answers, and next learning suggestion for topic '{last_topic}'."
    feedback = google_ai_chat(prompt)
    memory_bank[student_id]["last_feedback"] = feedback
    # Save memory
    with open("memory_bank.pkl", "wb") as f:
        pickle.dump(memory_bank, f)
    return feedback

# Example Usage
student_id = "student123"
topic = "Photosynthesis"

print("=== Topic Explanation ===")
print(explain_topic(student_id, topic))
print("\n=== Practice Questions ===")
questions = generate_questions(topic)
print(questions)
print("\n=== Feedback ===")
answers = ["Student answer 1", "Student answer 2", "Student answer 3"]
print(provide_feedback(student_id, answers))


=== Topic Explanation ===


AttributeError: module 'google.cloud.aiplatform' has no attribute 'TextGenerationModel'